In [ ]:
import pandas as pd
import numpy as np
import torch
import os
from scipy.io import loadmat
from imageio import imread
import matplotlib.pyplot as plt
import pickle
import data_utils
from torch.utils.data import Dataset

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
i = 3
imgs = os.listdir(data_utils.COAD_IMG_DIR)
current_img = data_utils.COAD_IMG_DIR+imgs[i]

In [ ]:
all_slides = []
all_slide_tile_classes = []
all_slide_locs = []
all_slide_labels = []
#all_slide_names = []

for i,name in enumerate(imgs):
    if name[0:3] == 'img':
        print(name, end=" ")
    
        current_img = data_utils.COAD_IMG_DIR+name
        slide_level_label,epi_mat,fibro_mat,inf_mat,other_mat,bmp_im = data_utils.process_slide(current_img,imgs,i)
        slide,slide_tile_class,slide_locs = data_utils.read_slide(epi_mat,fibro_mat,inf_mat,other_mat,bmp_im)
    
        all_slides.append(slide)
        all_slide_tile_classes.append(slide_tile_class)
        all_slide_locs.append(slide_locs)
        all_slide_labels.append(slide_level_label)
        #all_slide_names.append(current_img)

In [ ]:
pickle_file = data_utils.COAD_DATASET
with open(pickle_file, 'wb') as f: 
        pickle.dump([all_slides, all_slide_tile_classes, all_slide_locs, all_slide_labels], f)

pickle_file = data_utils.COAD_VALID
with open(pickle_file, 'wb') as f: 
        pickle.dump([all_slides[50:], all_slide_tile_classes[50:], all_slide_locs[50:], all_slide_labels[50:]], f)

pickle_file = data_utils.COAD_TRAIN
with open(pickle_file, 'wb') as f: 
        pickle.dump([all_slides[:50], all_slide_tile_classes[:50], all_slide_locs[:50], all_slide_labels[:50]], f)

pickle_file = data_utils.COAD_DEV
with open(pickle_file, 'wb') as f: 
        pickle.dump([all_slides[:10], all_slide_tile_classes[:10], all_slide_locs[:10], all_slide_labels[:10]], f)

# dataset too small
#pickle_file = data_utils.COAD_TEST
#with open(pickle_file, 'wb') as f: 
#        pickle.dump([all_slides[90:], all_slide_tile_classes[90:], all_slide_locs[90:],all_slide_labels[90:]], f)

In [ ]:
dev = data_utils.COAD_dataset(data_utils.COAD_DEV)

In [ ]:
dev_loader = torch.utils.data.DataLoader(dev, batch_size=1, shuffle=True, pin_memory=True)

In [ ]:
for slide,label in dev_loader:
    slide.squeeze_()
    break

In [ ]:
slide_level_label

In [1]:
temp = 'abcdefg.pt'
temp[:-3]

'abcdefg'